In [24]:
from collections import defaultdict
from itertools import count

with open('inputs/20.txt') as file:
    input = file.read()


In [33]:
%%time

maze = defaultdict(lambda:' ')
B,R = 0,0
for y,row in enumerate(input.split('\n')):
    for x,c in enumerate(row):
        maze[x,y] = c
        R = max(R,x)
        B = max(B,y)

IL,IT,IR,IB = R,B,0,0
for y in range(2,B-1):
    for x in range(2,R-1):
        if maze[x,y] == ' ':
            IL = min(IL,x)
            IR = max(IR,x)
            IT = min(IT,y)
            IB = max(IB,y)

portals = defaultdict(lambda:[])
for y in range(2,B-1):
    if maze[0,y]!=' ':
        portals[maze[0,y]+maze[1,y]].append((2,y))
    if maze[R,y]!=' ':
        portals[maze[R-1,y]+maze[R,y]].append((R-2,y))
for y in range(IT,IB+1):
    if maze[IL,y]!=' ':
        portals[maze[IL,y]+maze[IL+1,y]].append((IL-1,y))
    if maze[IR,y]!=' ':
        portals[maze[IR-1,y]+maze[IR,y]].append((IR+1,y))
for x in range(2,R-1):
    if maze[x,0]!=' ':
        portals[maze[x,0]+maze[x,1]].append((x,2))
    if maze[x,B]!=' ':
        portals[maze[x,B-1]+maze[x,B]].append((x,B-2))
for x in range(IL,IR+1):
    if maze[x,IT]!=' ':
        portals[maze[x,IT]+maze[x,IT+1]].append((x,IT-1))
    if maze[x,IB]!=' ':
        portals[maze[x,IB-1]+maze[x,IB]].append((x,IB+1))

links = {}
for pp in portals.values():
    if len(pp)>1:
        links[pp[0]]=pp[1]
        links[pp[1]]=pp[0]

def pretty():
    print()
    for y in range(B+1):
        row = ''
        for x in range(R+1):
            p = (x,y)
            v = maze[p]
            if p in links:
                row += '\x1b[31m+\x1b[0m'
                continue
            if p == portals['AA'][0]:
                row += '\x1b[32m+\x1b[0m'
                continue
            if p == portals['ZZ'][0]:
                row += '\x1b[34m+\x1b[0m'
                continue
            if v==' ':
                if x==IL or x==IR:
                    row += '|'
                    continue
                if y==IT or y==IB:
                    row += '-'
                    continue
            row += v
        print(row)
pretty()



                           |       G A     J     I         O       X       O   |                           
                           |       N V     N     P         H       N       B   |                           
  #################################+#+#####+#####+#########+#######+#######+#############################  
  #.....#.#.........#...#.......#.......#.#...#.........#.....#...#...#.....#.........#.....#.......#.#.#  
  #.###.#.#####.#.#####.#.#####.#####.###.#.###.#########.#####.#.###.#.#########.#######.#####.#.###.#.#  
  #.#.....#...#.#...#...#.#.#.......#.#.......#...#.#.#.......#.#.....#.......#.....#.#.....#.#.#.#.#...#  
  #####.#####.#####.###.###.###.###.#.#.#######.###.#.#.#.#####.#########.#######.#.#.#.#####.###.#.#.###  
  #.......#.#.#...#...#.........#...#.#...#...........#.#.#.........#.....#.......#.#.....#.......#...#.#  
  #.#.#####.#.###.#.###.#.###.#.###.#.#.###.###.#####.#.#####.#.#########.#.#.#######.###.###.###.#.###.#  
  #.#.#...#...#.#.......#..

In [36]:
%%time

def walk():
    dist = defaultdict(lambda:float('inf'))
    start = portals['AA'][0]
    goal = portals['ZZ'][0]
    next = [start]
    dist[start] = 0
    for step in count():
        print(step, next)
        if len(next)==0: break
        cur,next = next,[]
        for p in cur:
            print(p)
            for d in [(1,0),(-1,0),(0,1),(0,-1)]:
                np = (p[0]+d[0],p[1]+d[1])
                # print('checking',np)
                if maze[np]=='.' and dist[np]>step:
                    dist[np] = step
                    next.append(np)

walk()



0 [(71, 106)]
(71, 106)
checking (72, 106)
checking (70, 106)
checking (71, 107)
checking (71, 105)
1 [(71, 105)]
(71, 105)
checking (72, 105)
checking (70, 105)
checking (71, 106)
checking (71, 104)
2 [(70, 105), (71, 104)]
(70, 105)
checking (71, 105)
checking (69, 105)
checking (70, 106)
checking (70, 104)
(71, 104)
checking (72, 104)
checking (70, 104)
checking (71, 105)
checking (71, 103)
3 [(69, 105), (71, 103)]
(69, 105)
checking (70, 105)
checking (68, 105)
checking (69, 106)
checking (69, 104)
(71, 103)
checking (72, 103)
checking (70, 103)
checking (71, 104)
checking (71, 102)
4 [(68, 105), (70, 103)]
(68, 105)
checking (69, 105)
checking (67, 105)
checking (68, 106)
checking (68, 104)
(70, 103)
checking (71, 103)
checking (69, 103)
checking (70, 104)
checking (70, 102)
5 [(67, 105), (69, 103)]
(67, 105)
checking (68, 105)
checking (66, 105)
checking (67, 106)
checking (67, 104)
(69, 103)
checking (70, 103)
checking (68, 103)
checking (69, 104)
checking (69, 102)
6 [(66, 105)